In [4]:
PATH = 'Trust.csv'

In [5]:
import numpy as np
import pandas as pd
import logging
from joblib import Parallel, delayed

# Configure logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(levelname)s:%(message)s")

# Payoff matrix as provided in the original code
payoffs = np.array(
    [
        [
            [[-9231, -9231, -9231], [-9231, -9231, -13066]],
            [[-9231, -11943, -9231], [-9231, -5969, -5969]],
        ],
        [
            [[-10820, -9231, -9231], [-5969, -9231, -5969]],
            [[-5445, -5445, -9231], [-3857, -3857, -3857]],
        ],
    ]
)

def get_payoff(player_id, p1_strategy, p2_strategy, p3_strategy):
    return payoffs[p1_strategy, p2_strategy, p3_strategy][player_id]

def make_decision_based_on_trust(trust):
    return int(np.random.random() < trust)

def update_trust_state(n_interactions, n_successes, expected_outcome, actual_outcome):
    n_interactions += 1
    if expected_outcome == actual_outcome:
        n_successes += 1
    return n_interactions, n_successes

def play_1_game_with_trust_and_state(generator, trust1, trust2, trust3, n1_interactions, n1_successes, n2_interactions, n2_successes, n3_interactions, n3_successes):
    p1_strategy = make_decision_based_on_trust(trust1)
    p2_strategy = make_decision_based_on_trust(trust2)
    p3_strategy = make_decision_based_on_trust(trust3)
    
    # Assume each player expects the other two to collaborate
    p1_expected_outcome = 1
    p2_expected_outcome = 1
    p3_expected_outcome = 1

    # Update state variables based on actual outcomes
    n1_interactions, n1_successes = update_trust_state(n1_interactions, n1_successes, p1_expected_outcome, p2_strategy == 1 and p3_strategy == 1)
    n2_interactions, n2_successes = update_trust_state(n2_interactions, n2_successes, p2_expected_outcome, p1_strategy == 1 and p3_strategy == 1)
    n3_interactions, n3_successes = update_trust_state(n3_interactions, n3_successes, p3_expected_outcome, p1_strategy == 1 and p2_strategy == 1)
    
    # Recalculate trust based on updated state
    trust1 = n1_successes / n1_interactions if n1_interactions > 0 else 0
    trust2 = n2_successes / n2_interactions if n2_interactions > 0 else 0
    trust3 = n3_successes / n3_interactions if n3_interactions > 0 else 0
    
    return trust1, trust2, trust3, n1_interactions, n1_successes, n2_interactions, n2_successes, n3_interactions, n3_successes

def run_game_get_final_trust(
    trust1,
    trust2,
    trust3,
    seed,
    num_games,
    early_stop=True,
    epsilon=10e-20,
):
    generator = np.random.default_rng(seed)
    n1_interactions, n1_successes = 0, 0
    n2_interactions, n2_successes = 0, 0
    n3_interactions, n3_successes = 0, 0

    for i in range(num_games):
        trust1, trust2, trust3, n1_interactions, n1_successes, n2_interactions, n2_successes, n3_interactions, n3_successes = play_1_game_with_trust_and_state(
            generator, trust1, trust2, trust3, n1_interactions, n1_successes, n2_interactions, n2_successes, n3_interactions, n3_successes)

        if early_stop:
            delta = (
                abs(trust1 - trust1)
                + abs(trust2 - trust2)
                + abs(trust3 - trust3)
            )
            if delta < epsilon:
                break

    logging.debug(f"Number of Games {i+1}")

    return (trust1, trust2, trust3)

def compute_game(trust1, trust2, trust3, seed):
    try:
        task = trust1, trust2, trust3, seed
        trust_final = run_game_get_final_trust(
            trust1,
            trust2,
            trust3,
            seed=seed,
            num_games=3000,
            early_stop=True,
            epsilon=10e-20,
        )
        logging.debug(f"Task completed successfully for: {task}")
        return [
            seed,
            trust1,
            trust2,
            trust3,
            trust_final[0],
            trust_final[1],
            trust_final[2],
        ]
    except Exception as e:
        logging.error(f"Error in worker: {e} | Task: {task}")
        return None

results = Parallel(-1)(
    delayed(compute_game)(init_trust1, init_trust2, init_trust3, seed)
    for init_trust1 in np.linspace(0,1,11)
    for init_trust2 in np.linspace(0,1,11)
    for init_trust3 in np.linspace(0,1,11)
    for seed in range(10)
)

df = pd.DataFrame(results, columns=[
            "seed",
            "init_trust1",
            "init_trust2",
            "init_trust3",
            "final_trust1",
            "final_trust2",
            "final_trust3",
        ])

df.to_csv(PATH)

display(df)


,seed,init_trust1,init_trust2,init_trust3,final_trust1,final_trust2,final_trust3
0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
13305,5,1.0,1.0,1.0,1.0,1.0,1.0
13306,6,1.0,1.0,1.0,1.0,1.0,1.0
13307,7,1.0,1.0,1.0,1.0,1.0,1.0
13308,8,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
df.final_trust1.unique()

array([0., 1.])

In [7]:
df.final_trust2.unique()

array([0., 1.])

In [8]:
df.final_trust3.unique()

array([0., 1.])

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import numpy as np

data_path = PATH   
results_data_not_rounded = pd.read_csv(data_path)
results_data = results_data_not_rounded.round(1)

In [10]:

init_trust3_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fig, ax = plt.subplots(figsize=(10, 8))

# Initialize an empty heatmap with zeros
initial_heatmap_data = np.zeros((11, 11))
heatmap = sns.heatmap(initial_heatmap_data, cmap="viridis", vmin=0, vmax=1, ax=ax, cbar_kws={'label': 'Final Trust'})

def update(frame):
    init_trust3_value = init_trust3_values[frame]
    filtered_data = results_data[results_data['init_trust3'] == init_trust3_value]
    heatmap_data_trust1 = filtered_data.groupby(['init_trust1', 'init_trust2']).final_trust1.mean().reset_index()
    heatmap_trust1 = heatmap_data_trust1.pivot_table(index='init_trust2', columns='init_trust1', values='final_trust1')
    
    ax.clear()  
    sns.heatmap(heatmap_trust1, cmap="viridis", annot=True, vmin=0, vmax=1, ax=ax, cbar=False)  # Reuse the color bar
    ax.set_title(f'Heatmap of Final Trust for Player 1 (init_trust3 fixed at {init_trust3_value})')
    ax.set_xlabel('Initial Trust of Player 1')
    ax.set_ylabel('Initial Trust of Player 2')
    ax.invert_yaxis()  

ani = FuncAnimation(fig, update, frames=len(init_trust3_values), repeat=False)
plt.close(fig)

HTML(ani.to_jshtml())


2024-09-05 10:02:51,372 DEBUG:Loaded backend module://matplotlib_inline.backend_inline version unknown.
2024-09-05 10:02:51,373 DEBUG:Loaded backend module://matplotlib_inline.backend_inline version unknown.
2024-09-05 10:02:51,376 DEBUG:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
2024-09-05 10:02:51,376 DEBUG:findfont: score(FontEntry(fname='C:\\Users\\mzarreh\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\cmr10.ttf', name='cmr10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2024-09-05 10:02:51,376 DEBUG:findfont: score(FontEntry(fname='C:\\Users\\mzarreh\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizTwoSymBol.ttf', name='STIXSizeTwoSym', sty

In [14]:

init_trust3_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fig, ax = plt.subplots(figsize=(10, 8))

# Initialize an empty heatmap with zeros
initial_heatmap_data = np.zeros((11, 11))
heatmap = sns.heatmap(initial_heatmap_data, cmap="viridis", vmin=0, vmax=1, ax=ax, cbar_kws={'label': 'Final Trust'})

def update(frame):
    init_trust3_value = init_trust3_values[frame]
    filtered_data = results_data[results_data['init_trust3'] == init_trust3_value]
    heatmap_data_trust2 = filtered_data.groupby(['init_trust1', 'init_trust2']).final_trust2.mean().reset_index()
    heatmap_trust2 = heatmap_data_trust2.pivot_table(index='init_trust1', columns='init_trust2', values='final_trust2')
    
    ax.clear()  
    sns.heatmap(heatmap_trust2, cmap="viridis", annot=True, vmin=0, vmax=1, ax=ax, cbar=False)  # Reuse the color bar
    ax.set_title(f'Heatmap of Final Trust for Player 2 (init_trust3 fixed at {init_trust3_value})')
    ax.set_xlabel('Initial Trust of Player 2')
    ax.set_ylabel('Initial Trust of Player 1')
    ax.invert_yaxis()  

ani = FuncAnimation(fig, update, frames=len(init_trust3_values), repeat=False)
plt.close(fig)

HTML(ani.to_jshtml())


2024-09-05 10:14:21,135 DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x000001E9611B21B0>


2024-09-05 10:14:21,185 INFO:Animation.save using <class 'matplotlib.animation.HTMLWriter'>
2024-09-05 10:14:21,186 DEBUG:frame size in pixels is 1000 x 800
2024-09-05 10:14:23,164 DEBUG:MovieWriter: clearing temporary path=<TemporaryDirectory 'C:\\Users\\mzarreh\\AppData\\Local\\Temp\\tmpu0jov69k'>


In [15]:

init_trust2_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fig, ax = plt.subplots(figsize=(10, 8))

# Initialize an empty heatmap with zeros
initial_heatmap_data = np.zeros((11, 11))
heatmap = sns.heatmap(initial_heatmap_data, cmap="viridis", vmin=0, vmax=1, ax=ax, cbar_kws={'label': 'Final Trust'})

def update(frame):
    init_trust2_value = init_trust2_values[frame]
    filtered_data = results_data[results_data['init_trust2'] == init_trust2_value]
    heatmap_data_trust3 = filtered_data.groupby(['init_trust1', 'init_trust3']).final_trust3.mean().reset_index()
    heatmap_trust3 = heatmap_data_trust3.pivot_table(index='init_trust1', columns='init_trust3', values='final_trust3')
    
    ax.clear()  
    sns.heatmap(heatmap_trust3, cmap="viridis", annot=True, vmin=0, vmax=1, ax=ax, cbar=False)  
    ax.set_title(f'Heatmap of Final Trust for Player 3 (init_trust2 fixed at {init_trust2_value})')
    ax.set_xlabel('Initial Trust of Player 3')
    ax.set_ylabel('Initial Trust of Player 1')
    ax.invert_yaxis()  

ani = FuncAnimation(fig, update, frames=len(init_trust2_values), repeat=False)
plt.close(fig)

HTML(ani.to_jshtml())


2024-09-05 10:17:36,854 DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x000001E960F79820>
2024-09-05 10:17:36,896 INFO:Animation.save using <class 'matplotlib.animation.HTMLWriter'>
2024-09-05 10:17:36,896 DEBUG:frame size in pixels is 1000 x 800
2024-09-05 10:17:38,812 DEBUG:MovieWriter: clearing temporary path=<TemporaryDirectory 'C:\\Users\\mzarreh\\AppData\\Local\\Temp\\tmpkuqmb828'>
